# Data Science with Python: Time-series forecasting
### Martin John Madsen, Ph.D. <br/>Data Scientist @ [Allegion](www.allegion.com) <br/> [Connect with me on LinkedIn](https://www.linkedin.com/in/martinjohnmadsen/) <br/> Presented at [IndyPy, March 26, 2019](https://www.meetup.com/indypy/)

I walk through a Python data science project in this notebook and presentation. I cover the basic process of doing data science and building a machine learning model including:

1. Extracting data from a public API
2. Exploring the data in a Jupyter notebook
3. Building an ARIMA time-series forecast model for the data
4. Evaluating the model

Here are some additional resources that cover the process and the data I use:

* https://www.alkaline-ml.com/pmdarima/modules/classes.html
* https://github.com/tgsmith61591/pmdarima
* https://github.com/madsenmj/data-iowa-liquor
* https://github.com/madsenmj/ml-introduction-course
* https://otexts.com/fpp2/


## Why data science? Why time series? Why Python?

**Why data science?** I believe that everyone can and should be citizen scientists. I define data science as the process of using data to make predictions with the goal of using those predictions to make decisions. I believe that everyone can make better decisions by using data and models.

**Why time series?** This is a very common forecasting problem that shows up in a lot of places both in the business world as well as in many other contexts. Of all of the machine learning tools, this is probably the most useful for the average person.

**Why python?** Python has a low barrier to entry for first-time coders, it is accesable to lots of people, and it makes it easy to get started.

## Getting Started: Library Imports and Definitions

I use the [Anaconda Python 3.7 distribution](https://www.anaconda.com/distribution/). I like to use the [Jupyter notebook](https://docs.anaconda.com/ae-notebooks/user-guide/basic-tasks/apps/jupyter/) for exploring data and building machine learning models. It is flexible and powerful at the same time.

Once I have installed Ananconda and launched the notebook server, I import useful data libraries and define a helpful function for printing lists in the notebook. Each cell is run by highlighting the cell (clicking on it) and either clicking the `run` button above or pressing `Shift-Enter` to execute the cell. I recommend using Anaconda as the base Python installation and installing any needed libraries beyond that using `pip install`.

In [ ]:
# This package is not in the base Anaconda package list
try:
    import pmdarima
except:
    !pip install pmdarima

# imports and setup
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 1000)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import json, requests, getpass, datetime, warnings
from dateutil.relativedelta import relativedelta

from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima

def print_wrapped(data, ncols=3):
    """A helper function to wrap data columns for easier viewing."""
    nrows = len(data)
    labels = data.index
    n_split_rows = int(np.ceil(nrows/ncols))
    for r in range(0,nrows,ncols):
        for c in range(ncols):
            try:
                numstr = '{}'.format(data[r + c])
                tabs = [' '] * (20 - len(labels[r + c]) - len(numstr))
                print(labels[r + c] + "".join(tabs) + numstr, end='\t')
            except:
                pass
        print()

# Data Extraction

I use the [Iowa Liquor sales data](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy). 

> This dataset contains the spirits purchase information of Iowa Class “E” liquor licensees by product and date of purchase from January 1, 2012 to current.

I focus on the sales in each county in Iowa. My query to the Socrata data store returns the total sales in dollars and in liters for each day for each county. Because I aggreage by county, the total number of queries I make is below the limit for public access. This means I do not need to apply for and use a developer key to access the data.

I use a helper function that requests the data from the web API and converts it into a list of responses. I repeat the query, offsetting the results by the total, until I get all of the data from the API.

The [API documentation](https://dev.socrata.com/foundry/data.iowa.gov/spsw-4jax) describes how to form a query and how to do things like grouping, limits, and offsets. I have built a simple test query to explore what the data look like from the API.

In [ ]:
url = 'https://data.iowa.gov/resource/spsw-4jax.json'
options = '?$select=date,county,sum(sale_dollars),sum(sale_liters)&$group=date,county&$limit=10&$offset=1500'
myResponse = requests.get(url + options, verify=True)
print(myResponse.content.decode('utf-8'))

I wrap the API call into a function to call as many times as I need to get all of the data.

In [ ]:
def getCityDailyData(offset=0):
    """
    A function to pull the sales by county from the Iowa public database.
    
    Args:
        offset (optional, default 0): An offset to the records (if there are more than 50,000)
        
    Returns:
        A JSON array with the results.
        
    """
    
    url = 'https://data.iowa.gov/resource/spsw-4jax.json'
    selectQuery = '?$select=date,county,sum(sale_dollars),sum(sale_liters)&$group=date,county'

    limitQuery = '&$limit=50000'
    if (offset > 0):
        offset = '&$offset={}'.format(offset)
        query = url + selectQuery + limitQuery + offset
    else:
        query = url + selectQuery + limitQuery

    # Send the query to the API endpoint
    myResponse = requests.get(query, verify=True)

    jData=''
    # For successful API call, response code will be 200 (OK)
        
    try:
        if(myResponse.ok):
            jData = json.loads(myResponse.content.decode('utf-8'))
            print("The response contains {0} properties".format(len(jData)))
        else:
            print(myResponse.status_code)
            print(myResponse.headers)
            
    except:
        print(myResponse.status_code)
        print(myResponse.headers)
        
        
    return jData

In [ ]:
data_set1 = getCityDailyData()

Because the number of results is equal to the query limit, there may be additional results. I run the query a second time with an offset of `50000` to see if there are additional results. I repeat this as many times as necessary until I get a response where the number of results is less than the limit, indicating that I have exhausted the data store.

In [ ]:
data_set2 = getCityDailyData(50000)

The results count is less than the query limit, so I have all of the data. I convert both results into Pandas dataframes and join them together into a single dataframe. The [Pandas dataframe](https://pandas.pydata.org/) is a very flexible and powerful data structure for working with large amounts of data. 

In [ ]:
ildfs = pd.io.json.json_normalize(data_set1)
ildfs2 = pd.io.json.json_normalize(data_set2)
df = ildfs.append(ildfs2)

df.head()

I save the data to a file for future use so I don't have to extract it again from the source each time I want to explore the data.

In [ ]:
df.to_csv('raw_iowa_data.csv', index=False)

# Data Exploration

Now I look at the data! I load the CSV data back in from the saved file so I start from a consistent point each time I run this data exploration. 

Although this exploratory data analysis (or EDA), is not a structured process, I follow a few basic guidelines:

1. Understand (and clean up) the data types: make sure dates are dates and numbers are numbers
2. Understand (and clean up) any string fields: what are the value counts? Do they make sense?
3. Visualize the data: do numeric values make sense? Are the value counts reasonable?

I start with looking at the data types in the dataframe.

In [ ]:
df = pd.read_csv('raw_iowa_data.csv')
df.dtypes

I convert the date into a datetime object so I can use it as a date. I also use the `.head()` function of the dataframe to look at a sample of the first 5 rows. If something went wrong with the date conversion, I might see the issue in the dataframe sample.

In [ ]:
df['date'] = pd.to_datetime(df['date'])
print(df.dtypes)
df.head()

I am going to cut off any data from the current month - that way I will only have full months to work with in the data.

In [ ]:
current_month = datetime.datetime.now().strftime('%Y-%m-01')
print("Initial last date: {}".format(df['date'].max()))

# Filter rows using the date column
df = df[df['date'] < current_month]
print("After filter last date: {}".format(df['date'].max()))

The numeric fields are already in the correct format. What does the county data look like? I have worked with this dataset previously and found that this isn't always the cleanest data. The source data has either changed the county names or have other mistakes in the county names. Since I want to do a county-by-county analysis of the liquor sales, I need to make sure that the county names are all correct. I start by listing out the county names and a count of how many times they appear in the data.

In [ ]:
county_rows = df.groupby('county').date.count()
print_wrapped(county_rows, ncols=4)

This is, indeed, a mess.

I'll do the following to clean it up:

1. Make everything uppercase so they are the same
2. Fix the misspelled names
3. Get rid of "El Paso" - that isn't even in Iowa!

I'll look at the result after doing this.

In [ ]:
df['county'] = df['county'].str.upper()
df.dropna(inplace=True)

df.loc[df['county'] == 'BUENA VIST','county'] = 'BUENA VISTA'
df.loc[df['county'] == 'CERRO GORD','county'] = 'CERRO GORDO'
df.loc[df['county'] == 'OBRIEN','county'] = "O'BRIEN"
df.loc[df['county'] == 'POTTAWATTA','county'] = "POTTAWATTAMIE"
df = df.loc[df['county'] != 'EL PASO']

col_v2 = df.groupby('county').date.count()
print_wrapped(col_v2, ncols=4)

I've got one more clean-up to do. I want to look at montly sales, but the date is not aggregated that way. I create a new column that I label `month` that will let me aggregate the monthly sales based on that column. Again, I look at the first few rows to check that the new column is what I expected it to be.

In [ ]:
df['month'] = df['date'].apply(lambda x: x.strftime('%Y-%m-01'))
df['month'] = pd.to_datetime(df['month'])
df.head()

### Visualizing Data

I am ready to look for trends and patterns in the data. The list of counties is pretty long; I will start by aggregating the entire state's sales by month first, both in dollars and liters. I then graph the sales (in liters) over time to see what that looks like.

In [ ]:
state_data = df.groupby('month').sum()
state_data['sum_sale_liters'].plot(figsize=[15,5])
plt.ylabel('State-wide Sales [liters]')
plt.show()

It looks like there are regular patterns here! There is a nice tool from the python `statsmodels` library that takes a dataset like this and tries to break it up into parts that repeat, regularly increase, and any leftovers from that. A perfectly predictable function might be the sum of a trend component and a repeatable component like this:

In [ ]:
x = np.arange('2012-01', '2019-02', dtype='datetime64[M]')
xm = (x - np.min(x)).astype('int')
y = 2.3 + 0.2 * xm + np.cos(2 * np.pi * xm / 12)
fig, ax = plt.subplots(figsize=[15,5])
ax.plot(x,y)
ax.set_ylabel("Test Data")
plt.show()

I test the decomposition of this sample data using the library to make sure it makes sense.

In [ ]:
xy = pd.DataFrame(y,x, columns=['y'])
res = seasonal_decompose(xy, model='additive', two_sided=False)

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1)
ax1.set_ylabel('Trend')
res.seasonal.plot(ax=ax2)
ax2.set_ylabel('Seasonal')
res.resid.plot(ax=ax3)
ax3.set_ylabel('Residuals')
plt.tight_layout()

The decomposition pulled out the linear part and the repeating pattern just like it should. I apply this same method to the state-wide liquor sales data to see if there are repeatble patterns in the data.

In [ ]:
res = seasonal_decompose(state_data['sum_sale_liters'], model='additive', two_sided=False)

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1)
ax1.set_ylabel('Trend')
res.seasonal.plot(ax=ax2)
ax2.set_ylabel('Seasonal')
res.resid.plot(ax=ax3)
ax3.set_ylabel('Residuals')
plt.tight_layout()

This is promising. There is a definite annual sales pattern combined with a slow growth trend. The `Residuals` part is what is left over after adding up the other two - this describes the randomness of the data. A perfectly predictable dataset will have very little in the residuals table. The state-aggregated data has a residual that is almost a big as the `Seasonal` component! I expect that forecasts on this data will not be super accurate.

## Forecasting Total Liters Sold

The first step for any data science model using this type of time series data is to split the data into `training` and `testing` data sets based on a cut time. The `train` data will be before that time cut, `test` after. The reason for this is so that there is a way to check how well the model is working. I use the `train` dataset to train the model. In this case, I use that data to get the fit coefficients. The I use those fit coefficients to predict what the data will look like out in the future where I have the `test` data - what actually happened.

I start with a simple, single time cut. I use `2017-06-01` as the time cut -- that gives me about 66 months of training data and 18 months of test data.

My next step is to get a baseline model - the easiest model I can think of to compare against. In this case, I think that the simple "use last year's number" is a reasonable first step.

In [ ]:
state_data_train = state_data[state_data.index <= '2017-06-01']
state_data_test = state_data[state_data.index > '2017-06-01']

# Get the dates from the test data
test_months = state_data_test.reset_index(drop=False)['month']

# Offset them by a year
previous_year = test_months.apply(lambda x: x - relativedelta(years=1))

# Get the values from the previous year
last_year_preds = state_data.loc[previous_year, 'sum_sale_liters'].values

I visualize this simple model by plotting the actuals (round dots) and the "model" predictions (as 'x' markers).

In [ ]:
fig_1, ax = plt.subplots( figsize=(15,5))
clrs = sns.color_palette("husl", 5)

# #############################################################################
# Plot actual test vs. forecasts:
plt.plot(state_data_train.index, state_data_train['sum_sale_liters'], marker='o', c=clrs[0])

plt.plot(state_data_test.index, state_data_test['sum_sale_liters'], marker='o', c=clrs[1])

plt.plot(state_data_test.index, last_year_preds, linestyle='None', marker='x', c=clrs[2])

plt.title("Actual test samples vs. forecasts -- Last Year's Number")
plt.ylabel('State Total Sales [liters]')
plt.tight_layout()
plt.close()
fig_1

This model does pretty well, considering how simple it is. I'll now run the more complicated ARIMA model to compare it against.

The `auto_arima` function tests a range of possible models on the data, looking for the "best" model based on how well the model describes the training data. The idea is that, if the data are predictable, the model that best describes the training data will also describe the test data. I use the `trace=True` argument so that I can watch the progress of the model and see the performance of each attempt.

I expect that the ARIMA model will have a seasonal component, given the results from decomposition plots. That decomposition had a period of 12 months, so I add that element to the ARIMA model.

In [ ]:
# Fit a simple auto_arima model
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    state_arima = auto_arima(state_data_train['sum_sale_liters'], 
                                 error_action='ignore', 
                                 trace=True,
                                 seasonal=True, 
                                 m=12)

I examine the output of the model fit. This gives me the "best" coefficients and model parameters. Not every model has model parameters that make any sense, but I like to look at them as a check that the model is going in the right direction. For example, if the standard errors are big (bigger than the coefficients), that is an indication that the model didn't fit right.

In [ ]:
state_arima.summary()

The standard errors look reasonable - at least they are not bigger than the fit coefficients. The rest of the numbers do not mean much to me - I am a visual person and like to plot the results to get a sense of how well the model is working.

I visualize the forecast against what actually happened. I plot the forecast predictions as green `-` marks and plot the actuals as round dots. It is important to note that these predictions are just the mean value of the prediction interval, but that the real forecast spans the entire range of the confidence interval. I show the forecast confidence interval as a shaded region. 

A better forecast will have a smaller confidence interval - that is what I am looking for to evaluate the quality of the forecast.

In [ ]:
predictions = state_arima.predict(n_periods=state_data_test.shape[0], return_conf_int=True)
pred_mean = predictions[0]
pred_interval = predictions[1]

fig_1, ax = plt.subplots( figsize=(15,5))
clrs = sns.color_palette("husl", 5)

# #############################################################################
# Plot actual test vs. forecasts:
plt.plot(state_data_train.index, state_data_train['sum_sale_liters'], marker='o', c=clrs[0])

plt.plot(state_data_test.index, state_data_test['sum_sale_liters'], marker='o', c=clrs[1])

plt.plot(state_data_test.index, last_year_preds, linestyle='None', marker='x', c=clrs[2])

plt.plot(state_data_test.index, pred_mean, linestyle='None', marker='_', c=clrs[3])
plt.fill_between(state_data_test.index, pred_interval[:,0], pred_interval[:,1], alpha=0.2, facecolor=clrs[3])

plt.title('Actual test samples vs. forecasts -- 18 month forecast')
plt.ylabel('State Total Sales [liters]')
plt.tight_layout()
plt.close()
fig_1

One of the biggest draw-backs from using the simple "last-year's model" is that it is very hard to estimate the confidence interval of the model. But the ARIMA model gives me the confidence intervals and, thus, a better sense of how well them model performs.

My first observation is that the forecast confidence interval gets bigger the further out the forecast goes. This is similar to the hurricane forcasts- the forecast works relatively well in the near-term, but futher out it gets harder to know what will happen. The size of the spread goes from about $\pm 30\%$ to around $\pm 70\%$ for the test date range. I plot this to get a better sense of the range of confidence intervals.

In [ ]:
fig_1, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5))
ax1.bar(range(len(state_data_test.index)), (pred_interval[:,1] - pred_mean))
ax1.set_xticklabels(state_data_test.index.strftime('%Y-%m'), rotation=40)
ax1.set_ylabel("Forecast Confidence Interval [liters]")

ax2.bar(range(len(state_data_test.index)), (pred_interval[:,1] - pred_mean)/(pred_mean)*100)
ax2.set_xticklabels(state_data_test.index.strftime('%Y-%m'), rotation=40)
ax2.set_ylabel("Forecast Relative Confidence Interval (%)")
plt.show()

Although the forecast is pretty bad out 16 months, I don't really care about the forecast 16 months from now. What I really want is the forecast for next month, where each month uses what I already know to make the forecast. This is a different type of `train/test` split. Something more like this:

<img src="cv1-1.png">

Where the blue dots represent the data I know (historical), and the red dots represent the month I am trying to forecast (out a month from where I am now).

I set up a for loop to step through each month to do the new type of rolling `train/test` split:

* extract the historical data from before that month, 
* create a model to forecast out the next month, 
* forecast out two months from the end of the historical data
* store the forecast and confidence in an array

In [ ]:
start_date = datetime.datetime(2017,6,1)

r = relativedelta(state_data.index.max(), start_date)
n_months = r.months + r.years * 12 - 1

prediction_df = pd.DataFrame({'date':[],'actual':[],'pred':[],'pred_interval_low':[],'pred_interval_high':[]})

for months_from_start in range(n_months):
    current_split_month = (start_date + relativedelta(months=months_from_start)).strftime('%Y-%m-%d')
    
    
    state_data_train = state_data[state_data.index <= current_split_month]
    state_data_test = state_data[state_data.index > current_split_month]
    print("Splitting at {} with {} months in train and {} in test".format(current_split_month,
                                                                         len(state_data_train),
                                                                         len(state_data_test)))
    

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        state_arima = auto_arima(state_data_train['sum_sale_liters'], error_action='ignore', trace=False,
                          seasonal=True, m=12)
    
    predict_month = start_date + relativedelta(months=(months_from_start + 2))
    predictions = state_arima.predict(n_periods=2, return_conf_int=True)
    pred_mean = predictions[0][-1]
    pred_interval = predictions[1][-1]
    
    actual = state_data_test.loc[predict_month.strftime('%Y-%m-%d'),'sum_sale_liters']
    
    pred_interval_low = pred_interval[0]
    pred_interval_high = pred_interval[1]
    prediction_df = prediction_df.append({'date':predict_month,
                                          'actual':actual,
                                         'pred':pred_mean,
                                         'pred_interval_low':pred_interval_low,
                                         'pred_interval_high':pred_interval_high}, ignore_index=True)
    
prediction_df.set_index('date', inplace=True)


Now we can show the two-month-out forecast against the actuals. We should see that the prediction interval doesn't grow over time and that we track the actual a little better since we are looking at the two-month-out forecast.

In [ ]:
fig_2, ax = plt.subplots( figsize=(15,5))
clrs = sns.color_palette("husl", 5)

# #############################################################################
# Plot actual test vs. forecasts:
plt.plot(state_data.index, state_data['sum_sale_liters'], c=clrs[0])

plt.plot(prediction_df.index, prediction_df['actual'], marker='o', c=clrs[1])

plt.plot(prediction_df.index, prediction_df['pred'], marker='_', linestyle='None', c=clrs[3])
plt.fill_between(prediction_df.index, prediction_df['pred_interval_low'], prediction_df['pred_interval_high'], alpha=0.2, facecolor=clrs[3])

plt.title('Actual test samples vs. forecasts -- rolling two-month forecast')
plt.ylabel('Sales [liters]')
plt.tight_layout()
plt.close()
fig_2

In [ ]:
fig_1, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5))
ax1.bar(range(len(prediction_df.index)),
       (prediction_df['pred_interval_high'] - prediction_df['pred']))
ax1.set_xticklabels(prediction_df.index.strftime('%Y-%m'), rotation=40)
ax1.set_ylabel("Forecast Confidence Interval [±liters]")

ax2.bar(range(len(prediction_df.index)), 
        (prediction_df['pred_interval_high'] - prediction_df['pred'])/(prediction_df['pred'])*100)
ax2.set_xticklabels(prediction_df.index.strftime('%Y-%m'), rotation=40)
ax2.set_ylabel("Forecast Relative Confidence Interval (%)")
plt.suptitle("Rolling 2-month-out Forecast")
plt.show()

Not only does the forecast confidence interval not grow as it did when I ran a 16-month-out forecast, I see that the forecast confidence interval gets a little smaller as I have more data in the training set, improving the forecast by a little bit. 

The state-wide forecast is still not very good. But I saw from the initial data decomposition that the residuals were quite large. So the lack of forecast quality is expected.

The process of doing data science is very iterative. At this point in the analysis, I cycle back to the beginning and run a new exploratory analysis on the county-level data to see what is there.

## County-level forecasts

Instead of looking at the state-wide aggregation, I look instead at the sales by county.

In [ ]:
county_data = df.groupby(['month','county']).sum().reset_index(drop=False, level=1)

fig, ax = plt.subplots(figsize=[15,8])

for county in county_data['county'].unique():
    county_data.loc[county_data['county'] == county, 'sum_sale_liters'].plot(ax=ax, label=county)
    
ax.set_yscale("log", nonposy='clip')
ax.set_ylabel("County Sales [liters]")
plt.show()

There is a huge range here from 100 L/month to 100,000 L/month! What are the top counties in terms of consumption?

In [ ]:
df.groupby('county').sum().sort_values('sum_sale_liters', ascending=False).head()

I take a quick break from the data and switch to searching Google for more information about each of the counties.

* POLK: 2010 Population: 430k, major city: Des Moines
* LINN: 2010 Population: 211k, major city: Cedar Rapids
* SCOTT: 2010 Population: 165k, major city: Davenport
* JOHNSON: 2010 Population: 161k, major city: Iowa City
* BLACK HAWK: 2010 Population: 131k, major city: Waterloo

Looks like population has something to do with liquor sales. I would like to know the liquor sales per capita for each county. Before going down that route, I sample a couple of counties and see what the data look like.

I start with POLK county, showing both the time series plot and the seasonal decomposition.

In [ ]:
df_county = df[df['county'] == 'POLK'].groupby('month').sum()
df_county['sum_sale_liters'].plot(figsize=[15,5])
plt.ylabel("POLK County Sales [liters]")

res = seasonal_decompose(df_county['sum_sale_liters'], model='additive', two_sided=False)
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1)
ax1.set_ylabel('Trend')
res.seasonal.plot(ax=ax2)
ax2.set_ylabel('Seasonal')
res.resid.plot(ax=ax3)
ax3.set_ylabel('Residuals')
plt.tight_layout()

The seasonal component isn't very strong here - it is about the same size as the residuals. Not a great candidate for forecasting. How about a different county?

In [ ]:
df_county = df[df['county'] == 'DICKINSON'].groupby('month').sum()
df_county['sum_sale_liters'].plot(figsize=[15,5])
plt.ylabel("DICKINSON County Sales [liters]")

res = seasonal_decompose(df_county['sum_sale_liters'], model='additive', two_sided=False)
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1)
ax1.set_ylabel('Trend')
res.seasonal.plot(ax=ax2)
ax2.set_ylabel('Seasonal')
res.resid.plot(ax=ax3)
ax3.set_ylabel('Residuals')
plt.tight_layout()

This looks great! The seasonal piece is huge compared to the residuals. I run the forecast for this county to see what it looks like.

In [ ]:
def forecast_county(df_county):
    """A function to create the rolling 2-month-out forecast"""
    
    start_date = datetime.datetime(2017,6,1)

    r = relativedelta(df_county.index.max(), start_date)
    n_months = r.months + r.years * 12 - 1

    prediction_county = pd.DataFrame({'date':[],'actual':[],'pred':[],'pred_interval_low':[],'pred_interval_high':[]})

    for months_from_start in range(n_months):
        current_split_month = (start_date + relativedelta(months=months_from_start)).strftime('%Y-%m-%d') 

        df_county_train = df_county[df_county.index <= current_split_month]
        df_county_test = df_county[df_county.index > current_split_month]
        print("Splitting at {} with {} months in train and {} in test".format(current_split_month,
                                                                             len(df_county_train),
                                                                             len(df_county_test)))

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            dickinson_arima = auto_arima(df_county_train['sum_sale_liters'], error_action='ignore', trace=False,
                              seasonal=True, m=12)

        predict_month = start_date + relativedelta(months=(months_from_start + 2))
        predictions = dickinson_arima.predict(n_periods=2, return_conf_int=True)
        pred_mean = predictions[0][-1]
        pred_interval = predictions[1][-1]

        actual = df_county_test.loc[predict_month.strftime('%Y-%m-%d'),'sum_sale_liters']

        pred_interval_low = pred_interval[0]
        pred_interval_high = pred_interval[1]
        prediction_county = prediction_county.append({'date':predict_month,
                                              'actual':actual,
                                             'pred':pred_mean,
                                             'pred_interval_low':pred_interval_low,
                                             'pred_interval_high':pred_interval_high}, ignore_index=True)

    prediction_county.set_index('date', inplace=True)
    return prediction_county

In [ ]:
def show_plots(df_county, prediction_county, county_name=""):
    """A function to plot the rolling 2-month-out forecast results"""
    
    fig = plt.figure(figsize=(15,10))
    gs = gridspec.GridSpec(2, 2)
    ax1 = plt.subplot(gs[0, :])
    ax2 = plt.subplot(gs[1,0])
    ax3 = plt.subplot(gs[1,1])

    clrs = sns.color_palette("husl", 5)

    # #############################################################################
    # Plot actual test vs. forecasts:
    ax1.plot(df_county.index, df_county['sum_sale_liters'], c=clrs[0])

    ax1.plot(prediction_county.index, prediction_county['actual'], marker='o', c=clrs[1])

    ax1.plot(prediction_county.index, prediction_county['pred'], marker='_', linestyle='None', c=clrs[3])
    ax1.fill_between(prediction_county.index, prediction_county['pred_interval_low'], prediction_county['pred_interval_high'], alpha=0.2, facecolor=clrs[3])

    ax1.set_ylabel('{} County Sales [liters]'.format(county_name))

    ax2.bar(range(len(prediction_county.index)),
           (prediction_county['pred_interval_high'] - prediction_county['pred']))
    ax2.set_xticklabels(prediction_county.index.strftime('%Y-%m'), rotation=40)
    ax2.set_ylabel("Forecast Confidence Interval [±liters]")

    ax3.bar(range(len(prediction_county.index)), 
            (prediction_county['pred_interval_high'] - prediction_county['pred'])/(prediction_county['pred'])*100)
    ax3.set_xticklabels(prediction_df.index.strftime('%Y-%m'), rotation=40)
    ax3.set_ylabel("Forecast Relative Confidence Interval (%)")
    plt.suptitle("Rolling 2-month-out Forecast for {} County".format(county_name))
    plt.show()

In [ ]:
prediction_county = forecast_county(df_county)
show_plots(df_county, prediction_county, 'DICKINSON')

Although the forecast looked pretty good, the relative confidence intervals are not any better than (and for some months, much worse than) the state-wide forecast.

## Adding Features
### US Census Data 

I can use the US Census population estimates https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html to get a feel for how many people live in each county.

I need a developer key to make this happen: https://www.census.gov/developers/. Click on "Request a Key" to get the key. I'll use the notebook password function to use the key without exposing it.

In [ ]:
api_key = getpass.getpass()

In [ ]:
def getPopulation(year, api_key):
    """
    Query the US census database to get the populations for all the cities in our list
    
    Args:
        year: the year to get the approximate population
        api_key: the api_key to use
        
    Returns:
        A cleaned dictionary of 'COUNTYNAME':'POPULATION' key:value pairs.
    """
    
    url='http://api.census.gov/data/{}/pep/population?get=POP,GEONAME'.format(year)
    placeQuery='&for=county:*'
    stateQuery='&in=state:19'
    key='&key=' + api_key

    #Get population data
    query=url+placeQuery+stateQuery+key
    popdict = dict()
    myResponse = requests.get(query, verify=True)
    
    # For successful API call, response code will be 200 (OK)
    try:
        if(myResponse.ok):
            poparray = json.loads(myResponse.content.decode('utf-8'))
            popdict = {a[1].replace(' County, Iowa','').upper():int(a[0]) for a in poparray[1:]}
        else:
            print(myResponse.status_code)
            print(myResponse.headers)
    except:
        print(myResponse.status_code)
        print(myResponse.headers)
    
    return popdict

I use this function to get the 2017 census data and save it to file.

In [ ]:
population_dict = getPopulation(2017, api_key)
# Save the dictionary so we don't have to call the API again
json.dump(population_dict, open('USCensusData.json', 'w'))

In [ ]:
# Load back in the saved dictonary
population_dict = json.load(open('USCensusData.json','r'))

In [ ]:
# Add the county population for each row in the data
df['population'] = df['county'].apply(lambda x: population_dict[x])
df.head()

Now I can look at liquor sales per capita:

In [ ]:
consumption = df.groupby('county').\
    agg({'sum_sale_liters': 'sum', 'population': 'max'})
consumption['literpercapita'] = consumption['sum_sale_liters']/consumption['population']
consumption.sort_values('literpercapita', ascending=False).head(10)

What is going on in those top 4 counties?

* DICKINSON: There are major reservoir/lakes here - it must be a big vacation spot.
* CERRO GORDO: There is also a lake here, too.
* KOSSUTH: No excuses. They just must buy more liquor. There is a casino in the next county over - maybe there is some spill from that?

# Conclusion

I have shown the basic outline walking through a data science project in a Jupyter notebook. This included pulling data from public API sources, exploring the data, and modeling the time series data. 